# Notebook 03: Développement et Test des Agents Individuels et de leurs Outils

Ce notebook se concentre sur le test et la démonstration de chaque agent que nous avons défini dans `src/agents/agent_architectures.py`. Nous allons instancier chaque agent, lui soumettre des tâches spécifiques, et observer comment il utilise ses outils (définis dans `src/agents/tool_definitions.py`) et comment il génère ses réponses.

**Prérequis :**
* Avoir exécuté `00_setup_environment.ipynb` (environnement configuré, clés API dans `.env`).
* Pour tester `DocumentAnalysisAgent` efficacement, il est préférable d'avoir une base de données MongoDB peuplée via `01_data_ingestion_and_embedding.ipynb` (ou `scripts/run_ingestion.py`) et que le `RetrievalEngine` (utilisé par `knowledge_base_retrieval_tool`) soit fonctionnel.

In [1]:
import logging
import sys
from pathlib import Path
import os
import json # Pour un affichage lisible des sorties d'outils
from typing import Optional, List # Ajout pour robustesse

# --- Configuration du PYTHONPATH et chargement de .env ---
# NOTE IMPORTANTE SUR LE CWD (Current Working Directory) :
# La ligne suivante `project_root = Path().resolve().parent` suppose que le CWD du notebook
# est le dossier `/notebooks/`. Si vous avez configuré VS Code pour que le CWD
# soit la racine du projet (`cognitive-swarm-agents/`), alors `Path().resolve()`
# donnerait déjà la racine du projet, et vous devriez utiliser :
# project_root = Path().resolve()
# Vérifiez votre CWD avec `import os; print(os.getcwd())` ou `from pathlib import Path; print(Path().resolve())` pour confirmer.
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    print(f"Ajout de {project_root} au PYTHONPATH")

from dotenv import load_dotenv
# De même, si CWD est la racine du projet, dotenv_path serait `Path().resolve() / ".env"`
dotenv_path = project_root / ".env"
if dotenv_path.exists():
    load_dotenv(dotenv_path=dotenv_path)
    print(f"Variables d'environnement chargées depuis : {dotenv_path}")
else:
    print(f"ATTENTION: Fichier .env non trouvé à {dotenv_path}. Assurez-vous qu'il est à la racine du projet.")

from config.settings import settings
from config.logging_config import setup_logging

# Importer les fonctions de création d'agents
# get_llm n'est plus importé ici car non utilisé directement ; les fonctions create_..._agent l'utilisent en interne via llm_factory
from src.agents.agent_architectures import (
    create_research_planner_agent,
    create_arxiv_search_agent,
    create_document_analysis_agent,
    create_synthesis_agent
    # get_llm # Supprimé car non utilisé directement dans ce notebook et plus défini dans agent_architectures
)

# Importer les outils pour d'éventuels tests directs
from src.agents.tool_definitions import (
    arxiv_search_tool, 
    knowledge_base_retrieval_tool,
    document_deep_dive_analysis_tool # Assurer que cet outil est importé pour test
)

# Importer les types de messages LangChain
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage # SystemMessage peut être utile

setup_logging(level="INFO") # Mettre à DEBUG pour voir les étapes internes des agents
logger = logging.getLogger("nb_03_agent_testing")

# --- Vérification des prérequis pour les LLMs et Embeddings (logique existante conservée et vérifiée) ---
# Pour les agents génératifs (Planner, ArxivSearcher, DocAnalyzer, Synthesizer)
# et pour les outils qui pourraient utiliser un LLM (comme document_deep_dive_analysis_tool via CrewAI).
active_llm_provider = settings.DEFAULT_LLM_MODEL_PROVIDER.lower()
logger.info(f"Les agents et certaines outils (comme CrewAI) utiliseront le fournisseur LLM génératif configuré : '{active_llm_provider}'.")
if active_llm_provider == "openai":
    if not settings.OPENAI_API_KEY:
        logger.error(f"ERREUR : Le fournisseur LLM est 'openai', mais OPENAI_API_KEY n'est pas configurée. Les tests des agents/outils utilisant ce LLM échoueront probablement.")
elif active_llm_provider == "huggingface_api":
    if not settings.HUGGINGFACE_API_KEY:
        logger.error(f"ERREUR : Le fournisseur LLM est 'huggingface_api', mais HUGGINGFACE_API_KEY n'est pas configurée.")
    if not settings.HUGGINGFACE_REPO_ID: # Vérification ajoutée pour être complet
        logger.error(f"ERREUR : Le fournisseur LLM est 'huggingface_api', mais HUGGINGFACE_REPO_ID n'est pas configuré.")
elif active_llm_provider == "ollama":
    if not settings.OLLAMA_BASE_URL:
        logger.error(f"ERREUR : Le fournisseur LLM est 'ollama', mais OLLAMA_BASE_URL n'est pas configurée.")
    if not settings.OLLAMA_GENERATIVE_MODEL_NAME: # Vérification ajoutée
        logger.error(f"ERREUR : Le fournisseur LLM est 'ollama', mais OLLAMA_GENERATIVE_MODEL_NAME (pour les agents) n'est pas configuré.")

# Pour knowledge_base_retrieval_tool (qui utilise RetrievalEngine, qui utilise le provider d'embedding configuré)
active_embedding_provider = settings.DEFAULT_EMBEDDING_PROVIDER.lower()
logger.info(f"Le 'knowledge_base_retrieval_tool' (via RetrievalEngine) utilisera le fournisseur d'embedding : '{active_embedding_provider}'")
if active_embedding_provider == "openai": # Pour les embeddings de requête
    if not settings.OPENAI_API_KEY:
        logger.error(f"ERREUR : Le fournisseur d'embedding pour RetrievalEngine est 'openai', mais OPENAI_API_KEY n'est pas configurée. Le 'knowledge_base_retrieval_tool' échouera probablement lors de la vectorisation des requêtes.")
elif active_embedding_provider == "ollama": # Pour les embeddings de requête
    if not settings.OLLAMA_BASE_URL:
         logger.error(f"ERREUR : Le fournisseur d'embedding pour RetrievalEngine est 'ollama', mais OLLAMA_BASE_URL n'est pas configurée.")
    if not settings.OLLAMA_EMBEDDING_MODEL_NAME: # Ajout de cette vérification cruciale
        logger.error(f"ERREUR : Le fournisseur d'embedding pour RetrievalEngine est 'ollama', mais OLLAMA_EMBEDDING_MODEL_NAME n'est pas configuré.")
# Pour "huggingface" (local), aucune clé API spécifique n'est requise pour l'embedding des requêtes.

if not settings.MONGODB_URI: 
    logger.error("ERREUR : MONGODB_URI non trouvé. Le 'knowledge_base_retrieval_tool' (via RetrievalEngine) ne pourra pas se connecter à la base de données.")

# TAVILY_API_KEY n'est pas utilisé par les outils testés directement dans les cellules suivantes de ce notebook,
# mais la vérification est conservée pour information générale si settings.py le liste.
if "TAVILY_API_KEY" in settings.model_fields and not settings.TAVILY_API_KEY: 
    logger.warning("Clé API TAVILY (TAVILY_API_KEY) non configurée (non critique pour les tests de base de ce notebook).")

Variables d'environnement chargées depuis : /home/facetoface/cognitive-swarm-agents/.env
2025-06-02 22:53:03 - nb_03_agent_testing - INFO - Les agents et certaines outils (comme CrewAI) utiliseront le fournisseur LLM génératif configuré : 'ollama'.
2025-06-02 22:53:03 - nb_03_agent_testing - INFO - Le 'knowledge_base_retrieval_tool' (via RetrievalEngine) utilisera le fournisseur d'embedding : 'ollama'
2025-06-02 22:53:03 - nb_03_agent_testing - WARNING - Clé API TAVILY (TAVILY_API_KEY) non configurée (non critique pour les tests de base de ce notebook).


/tmp/ipykernel_25398/3028106793.py:93: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  if "TAVILY_API_KEY" in settings.model_fields and not settings.TAVILY_API_KEY:


### 1. Test du `ResearchPlannerAgent`

Cet agent est conçu pour prendre une requête utilisateur complexe et la décomposer en un plan de recherche structuré. Il n'utilise pas d'outils pour cette tâche, se basant uniquement sur ses instructions (prompt système) et le LLM.

In [2]:
logger.info("--- Test du ResearchPlannerAgent ---")
# La première cellule de ce notebook (ID a57d7cbb) logue déjà des avertissements 
# si la configuration pour le active_llm_provider (DEFAULT_LLM_MODEL_PROVIDER) est manquante.
# Ce test tentera d'utiliser ce provider configuré.
logger.info(f"Tentative d'utilisation du provider LLM configuré par défaut: '{settings.DEFAULT_LLM_MODEL_PROVIDER}' pour le ResearchPlannerAgent.")

try:
    # create_research_planner_agent() appelle get_llm() qui utilise 
    # settings.DEFAULT_LLM_MODEL_PROVIDER et gère la configuration.
    # Une ValueError sera levée par get_llm si la configuration est incorrecte pour le provider choisi.
    planner_agent_executor = create_research_planner_agent() 
    
    # Requête utilisateur d'exemple (inchangée)
    user_query_plan = "What are the latest trends and key challenges in applying deep reinforcement learning to multi-robot navigation and coordination, particularly for swarm robotics?"
    logger.info(f"Requête pour le planificateur : '{user_query_plan}'")
    
    # Invocation de l'agent
    response_planner = planner_agent_executor.invoke({
        "messages": [HumanMessage(content=user_query_plan)]
    })
    research_plan = response_planner.get("output")
    
    print("\n--- Plan de Recherche Généré ---")
    if research_plan:
        print(research_plan)
    else:
        print("L'agent planificateur n'a pas retourné de plan (vérifiez les logs).")
        # Afficher la réponse complète peut aider au débogage si 'output' est vide mais la réponse existe
        print(f"Réponse complète de l'agent (si output est vide) : {response_planner}")
            
except ValueError as ve:
    # Attrape spécifiquement les erreurs de configuration de get_llm()
    logger.error(f"Erreur de configuration LLM pour le ResearchPlannerAgent (provider: {settings.DEFAULT_LLM_MODEL_PROVIDER}): {ve}", exc_info=True)
    print(f"ERREUR de configuration pour ResearchPlannerAgent : Le provider LLM '{settings.DEFAULT_LLM_MODEL_PROVIDER}' n'est pas correctement configuré (ex: clé API ou URL manquante). Détails : {ve}")
except Exception as e:
    # Attrape les autres erreurs pendant l'invocation de l'agent
    logger.error(f"Erreur lors de l'invocation du ResearchPlannerAgent: {e}", exc_info=True)
    print(f"Erreur inattendue lors du test du ResearchPlannerAgent: {e}")

# L'ancien bloc 'else' qui vérifiait settings.OPENAI_API_KEY est supprimé,
# car la logique try/except ci-dessus est plus générale et correcte.

2025-06-02 22:53:03 - nb_03_agent_testing - INFO - --- Test du ResearchPlannerAgent ---
2025-06-02 22:53:03 - nb_03_agent_testing - INFO - Tentative d'utilisation du provider LLM configuré par défaut: 'ollama' pour le ResearchPlannerAgent.
2025-06-02 22:53:03 - src.llm_services.llm_factory - INFO - Initializing LLM from llm_factory for provider: 'ollama' with temperature: 0.0
2025-06-02 22:53:03 - src.llm_services.llm_factory - INFO - Using Ollama model (via langchain_ollama): mistral from http://localhost:11434
2025-06-02 22:53:03 - src.agents.agent_architectures - INFO - Research Planner Agent created.
2025-06-02 22:53:03 - nb_03_agent_testing - INFO - Requête pour le planificateur : 'What are the latest trends and key challenges in applying deep reinforcement learning to multi-robot navigation and coordination, particularly for swarm robotics?'
2025-06-02 22:53:05 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"

--- Plan de Recherche Généré ---


### 2. Test du `ArxivSearchAgent` (avec `arxiv_search_tool`)

Cet agent est spécialisé dans la recherche d'articles sur ArXiv. Il utilise `arxiv_search_tool`.

In [3]:
logger.info("\n--- Test du ArxivSearchAgent ---")
logger.info(f"Tentative d'utilisation du provider LLM configuré par défaut: '{settings.DEFAULT_LLM_MODEL_PROVIDER}' pour l'ArxivSearchAgent.")

try:
    # create_arxiv_search_agent() appelle get_llm() qui utilise settings.DEFAULT_LLM_MODEL_PROVIDER
    arxiv_agent_executor = create_arxiv_search_agent()
    
    search_task = "Find 2 recent papers (last 6 months) on 'explainable reinforcement learning in robotics' sorted by submission date."
    logger.info(f"Tâche pour l'agent de recherche ArXiv : '{search_task}'")
    
    response_arxiv_search = arxiv_agent_executor.invoke({
        "messages": [HumanMessage(content=search_task)]
    })
    
    arxiv_results_output = response_arxiv_search.get("output")
    
    print("\n--- Résultats de la Recherche ArXiv (via Agent) ---")
    if arxiv_results_output:
        # La sortie de arxiv_search_tool est une liste de dictionnaires (ou un dict d'erreur)
        # Essayons de l'afficher de manière lisible
        try:
            # Si la sortie est une chaîne JSON, la parser. Sinon, l'afficher telle quelle.
            # L'outil retourne une liste de dicts, mais l'agent LLM pourrait la wrapper en chaîne.
            if isinstance(arxiv_results_output, str):
                try:
                    # Remplacer les apostrophes simples par des guillemets doubles pour une meilleure compatibilité JSON
                    # Attention: ceci est une heuristique et pourrait ne pas marcher pour tous les cas.
                    # Une meilleure solution serait que l'agent retourne directement un objet JSON valide ou un type de données structuré.
                    corrected_json_string = arxiv_results_output.replace("'", "\"") 
                    # Gérer les booléens Python True/False qui ne sont pas valides en JSON (true/false)
                    corrected_json_string = corrected_json_string.replace("True", "true").replace("False", "false")
                    # Gérer None qui n'est pas valide en JSON (null)
                    corrected_json_string = corrected_json_string.replace("None", "null")

                    parsed_output = json.loads(corrected_json_string)
                    print(json.dumps(parsed_output, indent=2, ensure_ascii=False))
                except json.JSONDecodeError:
                    print(f"Impossible de parser la sortie comme JSON. Sortie brute :\n{arxiv_results_output}")
            elif isinstance(arxiv_results_output, list) or isinstance(arxiv_results_output, dict): # Si c'est déjà un objet Python
                print(json.dumps(arxiv_results_output, indent=2, ensure_ascii=False))
            else: # Autre type, afficher directement
                 print(arxiv_results_output)
        except Exception as e_print:
            print(f"Erreur lors de l'affichage formaté des résultats : {e_print}")
            print("Sortie brute de l'agent :")
            print(arxiv_results_output)
    else:
        print("L'agent de recherche ArXiv n'a pas retourné de sortie (vérifiez les logs).")
        print(f"Réponse complète de l'agent (si output est vide) : {response_arxiv_search}")

    # Affichage des étapes intermédiaires (si disponibles et utiles)
    if "intermediate_steps" in response_arxiv_search and response_arxiv_search["intermediate_steps"]:
        print("\nÉtapes intermédiaires de l'agent ArXiv:")
        for step in response_arxiv_search["intermediate_steps"]:
            # La structure de 'step' peut varier selon le type d'agent (AgentAction vs ToolCall)
            if hasattr(step[0], 'tool') and hasattr(step[0], 'tool_input'): # Pour AgentAction
                tool_call_info = f"Outil: {step[0].tool}, Input: {step[0].tool_input}"
            elif isinstance(step[0], dict) and 'tool' in step[0] and 'tool_input' in step[0]: # Autre format possible
                tool_call_info = f"Outil: {step[0]['tool']}, Input: {step[0]['tool_input']}"
            else: # Fallback
                tool_call_info = str(step[0])

            tool_result = step[1]
            print(f"  Appel Outil: {tool_call_info}")
            print(f"  Résultat Outil (extrait): {str(tool_result)[:300]}...")
            
except ValueError as ve:
    logger.error(f"Erreur de configuration LLM pour l'ArxivSearchAgent (provider: {settings.DEFAULT_LLM_MODEL_PROVIDER}): {ve}", exc_info=True)
    print(f"ERREUR de configuration pour ArxivSearchAgent : Le provider LLM '{settings.DEFAULT_LLM_MODEL_PROVIDER}' n'est pas correctement configuré. Détails : {ve}")
except Exception as e:
    logger.error(f"Erreur lors de l'invocation de l'ArxivSearchAgent: {e}", exc_info=True)
    print(f"Erreur inattendue lors du test de l'ArxivSearchAgent: {e}")

2025-06-02 22:53:14 - nb_03_agent_testing - INFO - 
--- Test du ArxivSearchAgent ---
2025-06-02 22:53:14 - nb_03_agent_testing - INFO - Tentative d'utilisation du provider LLM configuré par défaut: 'ollama' pour l'ArxivSearchAgent.
2025-06-02 22:53:14 - src.llm_services.llm_factory - INFO - Initializing LLM from llm_factory for provider: 'ollama' with temperature: 0.0
2025-06-02 22:53:14 - src.llm_services.llm_factory - INFO - Using Ollama model (via langchain_ollama): mistral from http://localhost:11434
2025-06-02 22:53:14 - src.agents.agent_architectures - INFO - ArXiv Search Agent created with tools: ['arxiv_search_tool']
2025-06-02 22:53:14 - nb_03_agent_testing - INFO - Tâche pour l'agent de recherche ArXiv : 'Find 2 recent papers (last 6 months) on 'explainable reinforcement learning in robotics' sorted by submission date.'
2025-06-02 22:53:15 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
2025-06-02 22:53:15 - src.agents.tool_definitions - 

### 2b. Test Direct de `arxiv_search_tool` (Optionnel)

Pour mieux comprendre ce que l'outil `arxiv_search_tool` retourne, nous pouvons l'appeler directement.

In [4]:
logger.info("\n--- Test Direct de arxiv_search_tool ---")
try:
    direct_tool_results = arxiv_search_tool.invoke({
        "query": "transformer models for robot control", 
        "max_results": 1,
        "sort_by": "submittedDate"
    })
    print("\nRésultats Directs de arxiv_search_tool:")
    # Ajout de ensure_ascii=False pour un meilleur affichage des caractères spéciaux
    print(json.dumps(direct_tool_results, indent=2, ensure_ascii=False))
except Exception as e:
    logger.error(f"Erreur lors de l'appel direct à arxiv_search_tool: {e}", exc_info=True)
    print(f"Erreur lors de l'appel direct à arxiv_search_tool: {e}")

2025-06-02 22:53:27 - nb_03_agent_testing - INFO - 
--- Test Direct de arxiv_search_tool ---
2025-06-02 22:53:27 - src.agents.tool_definitions - INFO - Executing arxiv_search_tool with query='transformer models for robot control', max_results=1, sort_by='submittedDate', sort_order='descending'
2025-06-02 22:53:27 - arxiv - INFO - Requesting page (first: True, try: 0): https://export.arxiv.org/api/query?search_query=transformer+models+for+robot+control&id_list=&sortBy=submittedDate&sortOrder=descending&start=0&max_results=100
2025-06-02 22:53:30 - arxiv - INFO - Got first page: 100 of 2388778 total results
2025-06-02 22:53:30 - src.agents.tool_definitions - INFO - arxiv_search_tool found 1 papers.

Résultats Directs de arxiv_search_tool:
[
  {
    "entry_id": "http://arxiv.org/abs/2505.24878v1",
    "title": "Open CaptchaWorld: A Comprehensive Web-based Platform for Testing and Benchmarking Multimodal LLM Agents",
    "authors": [
      "Yaxin Luo",
      "Zhaoyi Li",
      "Jiacheng Li

### 3. Test du `DocumentAnalysisAgent` (avec `knowledge_base_retrieval_tool`)

Cet agent analyse les documents récupérés de notre base de connaissances (MongoDB). Son bon fonctionnement dépend de la présence de données pertinentes dans la collection (par exemple, `{COLLECTION_NAME_FOR_RAG_TEST}`) et de la fonctionnalité du `RetrievalEngine`.

In [5]:
logger.info("\n--- Test du DocumentAnalysisAgent ---")
logger.info(f"Tentative d'utilisation du provider LLM configuré par défaut: '{settings.DEFAULT_LLM_MODEL_PROVIDER}' pour le DocumentAnalysisAgent.")
logger.info(f"Ce test dépend aussi du provider d'embedding configuré: '{settings.DEFAULT_EMBEDDING_PROVIDER}' (pour knowledge_base_retrieval_tool via RetrievalEngine) et de MongoDB.")

# La connexion à MongoDB est essentielle pour les outils de cet agent.
if not settings.MONGODB_URI:
    logger.error("MONGODB_URI non configuré. Test du DocumentAnalysisAgent sauté car ses outils (knowledge_base_retrieval_tool) en dépendent crucialement.")
    print("ERREUR: MONGODB_URI non configuré dans .env. Le DocumentAnalysisAgent ne peut pas être testé correctement sans accès à la base de données.")
else:
    try:
        # La création de l'agent peut échouer si get_llm() (appelé en interne) échoue 
        # en raison d'une configuration manquante pour le DEFAULT_LLM_MODEL_PROVIDER.
        # De même, RetrievalEngine (utilisé par knowledge_base_retrieval_tool) peut échouer à l'init
        # si DEFAULT_EMBEDDING_PROVIDER est mal configuré.
        doc_analysis_agent_executor = create_document_analysis_agent()
        
        # Tâche d'analyse exemple.
        # Le DocumentAnalysisAgent est maintenant équipé du document_deep_dive_analysis_tool.
        # Son prompt système (DOCUMENT_ANALYSIS_SYSTEM_PROMPT_V2) le guide sur quand utiliser quel outil.
        # Voici une tâche qui devrait utiliser knowledge_base_retrieval_tool :
        analysis_task = "Based on the knowledge base, what are some common techniques for achieving explainability in RL agents used in robotics? Cite any relevant ArXiv IDs if found."
        
        # Pour tester le document_deep_dive_analysis_tool, la tâche devrait être formulée différemment,
        # par exemple, en demandant explicitement une "deep dive" sur un document spécifique.
        # Exemple :
        # analysis_task = "Perform a detailed, structured deep dive analysis on document 'XXXX.YYYYY' (you will need to fetch its content first if not provided) focusing on its methodology and limitations for sim-to-real transfer."
        # Pour un tel test, assurez-vous que le document 'XXXX.YYYYY' existe ou que l'agent peut le récupérer.
        
        logger.info(f"Tâche pour l'agent d'analyse de documents : '{analysis_task}'")
        
        response_doc_analysis = doc_analysis_agent_executor.invoke({
            "messages": [HumanMessage(content=analysis_task)]
        })
        analysis_output = response_doc_analysis.get("output")
        
        print("\n--- Résultats de l'Analyse de Documents (via Agent) ---")
        if analysis_output:
            # La sortie peut être une simple chaîne ou un rapport structuré (ex: de CrewAI via le deep_dive_tool)
            if isinstance(analysis_output, str) and analysis_output.lower().startswith("error:"):
                print(f"L'agent ou un outil a retourné une ERREUR gérée : {analysis_output}")
            else:
                # Tenter d'afficher en Markdown si la sortie est un rapport formaté (souvent le cas pour CrewAI)
                # ou en JSON si c'est une structure de données, sinon en texte brut.
                if isinstance(analysis_output, str) and ("\n## " in analysis_output or "\n### " in analysis_output or "```" in analysis_output) : # Heuristique pour Markdown
                    try:
                        from IPython.display import display, Markdown
                        print("Affichage de la sortie comme Markdown:")
                        display(Markdown(analysis_output))
                    except ImportError:
                        print(analysis_output) # Fallback
                elif isinstance(analysis_output, str) and analysis_output.strip().startswith(("{", "[")): # Heuristique pour JSON
                    try:
                        parsed_json = json.loads(analysis_output)
                        print("Sortie formatée comme JSON:")
                        print(json.dumps(parsed_json, indent=2, ensure_ascii=False))
                    except json.JSONDecodeError:
                        print(analysis_output) # Fallback
                else: # Si c'est déjà un dict/list ou une chaîne simple
                    if isinstance(analysis_output, (dict, list)):
                        print(json.dumps(analysis_output, indent=2, ensure_ascii=False))
                    else:
                        print(analysis_output)
        else:
            print("L'agent d'analyse de documents n'a pas retourné de sortie ('output' est vide/None).")
            print(f"Réponse complète de l'agent (pour débogage) : {response_doc_analysis}")

        # Affichage des étapes intermédiaires
        if "intermediate_steps" in response_doc_analysis and response_doc_analysis["intermediate_steps"]:
            print("\nÉtapes intermédiaires de l'agent d'Analyse:")
            for step in response_doc_analysis["intermediate_steps"]:
                tool_call_info = "N/A"
                # La structure de step[0] (l'action) peut varier.
                action_part = step[0]
                if hasattr(action_part, 'tool') and hasattr(action_part, 'tool_input'): # AgentAction
                    tool_call_info = f"Outil: {action_part.tool}, Input: {action_part.tool_input}"
                elif isinstance(action_part, list) and action_part and isinstance(action_part[0], dict) and 'tool_name' in action_part[0]: # Possible format pour tool_calls multiples
                     tool_call_info = f"Outil(s): {', '.join([tc.get('tool_name', 'unknown') for tc in action_part])}"
                elif isinstance(action_part, dict) and 'tool' in action_part and 'tool_input' in action_part: # Autre format possible
                     tool_call_info = f"Outil: {action_part['tool']}, Input: {action_part['tool_input']}"
                else: # Fallback
                    tool_call_info = str(action_part)

                tool_result = step[1] # Observation
                print(f"  Appel Outil/Action: {tool_call_info}")
                print(f"  Résultat Outil (extrait): {str(tool_result)[:300]}...")
                
    except ValueError as ve:
        logger.error(f"Erreur de configuration ou de valeur lors de la création/invocation du DocumentAnalysisAgent. Vérifiez les configurations pour le provider LLM '{settings.DEFAULT_LLM_MODEL_PROVIDER}' et le provider d'embedding '{settings.DEFAULT_EMBEDDING_PROVIDER}'. Détails: {ve}", exc_info=True)
        print(f"ERREUR (DocumentAnalysisAgent): Problème de configuration LLM ou Embedding. Provider LLM: '{settings.DEFAULT_LLM_MODEL_PROVIDER}', Provider Embedding: '{settings.DEFAULT_EMBEDDING_PROVIDER}'. Détails: {ve}")
    except Exception as e:
        logger.error(f"Erreur lors de l'invocation du DocumentAnalysisAgent: {e}", exc_info=True)
        print(f"Erreur inattendue lors du test du DocumentAnalysisAgent: {e}. Assurez-vous que RetrievalEngine peut s'initialiser, que MongoDB est accessible et peuplé, et que les configurations LLM/Embedding sont correctes.")

2025-06-02 22:53:30 - nb_03_agent_testing - INFO - 
--- Test du DocumentAnalysisAgent ---
2025-06-02 22:53:30 - nb_03_agent_testing - INFO - Tentative d'utilisation du provider LLM configuré par défaut: 'ollama' pour le DocumentAnalysisAgent.
2025-06-02 22:53:30 - nb_03_agent_testing - INFO - Ce test dépend aussi du provider d'embedding configuré: 'ollama' (pour knowledge_base_retrieval_tool via RetrievalEngine) et de MongoDB.
2025-06-02 22:53:30 - src.llm_services.llm_factory - INFO - Initializing LLM from llm_factory for provider: 'ollama' with temperature: 0.0
2025-06-02 22:53:30 - src.llm_services.llm_factory - INFO - Using Ollama model (via langchain_ollama): mistral from http://localhost:11434
2025-06-02 22:53:30 - src.agents.agent_architectures - INFO - Document Analysis Agent created with tools: ['knowledge_base_retrieval_tool', 'document_deep_dive_analysis_tool']
2025-06-02 22:53:30 - nb_03_agent_testing - INFO - Tâche pour l'agent d'analyse de documents : 'Based on the knowle

### 4. Test du `SynthesisAgent`

Cet agent prend des informations analysées (que nous allons simuler ici) et produit une synthèse structurée. Il n'utilise pas d'outils de récupération.

In [6]:
logger.info("\n--- Test du SynthesisAgent ---")
# create_synthesis_agent() dans agent_architectures.py appelle get_llm() 
# avec SYNTHESIS_LLM_TEMPERATURE depuis llm_factory.py.
# Il utilisera le settings.DEFAULT_LLM_MODEL_PROVIDER.
logger.info(f"Tentative d'utilisation du provider LLM configuré par défaut: '{settings.DEFAULT_LLM_MODEL_PROVIDER}' pour le SynthesisAgent.")

try:
    # La création de l'agent peut échouer si get_llm() (appelé en interne) échoue 
    # en raison d'une configuration manquante pour le DEFAULT_LLM_MODEL_PROVIDER.
    synthesis_agent_executor = create_synthesis_agent()
    
    # Préparer un contexte simulé (inchangé par rapport à la version originale de la cellule)
    simulated_context_for_synthesis = """
    User Query: What are key considerations for sim-to-real transfer in robotic reinforcement learning?

    Information from Document Analysis:
    - Chunk from ArXiv ID 123.4567: Sim-to-real transfer often suffers from domain randomization issues. Techniques like domain adaptation and system identification are crucial. Physical parameters like friction and sensor noise are hard to model accurately.
    - Chunk from ArXiv ID 789.0123: Using realistic simulators and adding noise during training can improve transfer. Photorealistic rendering helps vision-based policies. Policy distillation from an ensemble of simulation-trained agents is a promising approach.
    - ArXiv Search found paper 'Recent Advances in Sim-to-Real for Robotics' (ArXiv:2401.0001), summary: This paper reviews state-of-the-art methods, highlighting the importance of robust learning algorithms and accurate dynamics modeling.
    """
    
    # Le message pour l'agent de synthèse inclut maintenant le contexte directement.
    # Le prompt de l'agent de synthèse (SYNTHESIS_AGENT_SYSTEM_PROMPT) lui indique de travailler
    # avec les informations fournies dans les messages.
    synthesis_task_message = HumanMessage(
        content=f"Based on the provided information below, write a concise summary report on the key considerations for sim-to-real transfer in robotic reinforcement learning. The report should be well-structured and directly address the initial user query mentioned in the context.\n\nProvided Information:\n{simulated_context_for_synthesis}"
    )
    logger.info("Tâche pour l'agent de synthèse (basée sur un contexte simulé).")

    response_synthesis = synthesis_agent_executor.invoke({
        "messages": [synthesis_task_message] # L'historique des messages peut aussi être passé si pertinent
    })
    synthesized_output = response_synthesis.get("output")
    
    print("\n--- Sortie de Synthèse (via Agent) ---")
    if synthesized_output:
        # Le SynthesisAgent est censé produire du texte formaté (potentiellement Markdown).
        if isinstance(synthesized_output, str) and (synthesized_output.lower().startswith("error:") or "erreur" in synthesized_output.lower()):
             print(f"L'agent de synthèse semble avoir rencontré une difficulté : {synthesized_output}")
        else:
            try:
                from IPython.display import display, Markdown
                display(Markdown(str(synthesized_output)))
            except ImportError:
                print(str(synthesized_output))
    else:
        print("L'agent de synthèse n'a pas retourné de sortie ('output' est vide/None).")
        print(f"Réponse complète de l'agent (pour débogage) : {response_synthesis}")
            
except ValueError as ve:
    logger.error(f"Erreur de configuration LLM pour le SynthesisAgent (provider: {settings.DEFAULT_LLM_MODEL_PROVIDER}): {ve}", exc_info=True)
    print(f"ERREUR de configuration pour SynthesisAgent : Le provider LLM '{settings.DEFAULT_LLM_MODEL_PROVIDER}' n'est pas correctement configuré. Détails : {ve}")
except Exception as e:
    logger.error(f"Erreur lors de l'invocation du SynthesisAgent: {e}", exc_info=True)
    print(f"Erreur inattendue lors du test du SynthesisAgent: {e}")

2025-06-02 22:53:40 - nb_03_agent_testing - INFO - 
--- Test du SynthesisAgent ---
2025-06-02 22:53:40 - nb_03_agent_testing - INFO - Tentative d'utilisation du provider LLM configuré par défaut: 'ollama' pour le SynthesisAgent.
2025-06-02 22:53:40 - src.llm_services.llm_factory - INFO - Initializing LLM from llm_factory for provider: 'ollama' with temperature: 0.5
2025-06-02 22:53:40 - src.llm_services.llm_factory - INFO - Using Ollama model (via langchain_ollama): mistral from http://localhost:11434
2025-06-02 22:53:40 - src.agents.agent_architectures - INFO - Synthesis Agent created.
2025-06-02 22:53:40 - nb_03_agent_testing - INFO - Tâche pour l'agent de synthèse (basée sur un contexte simulé).
2025-06-02 22:53:40 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"

--- Sortie de Synthèse (via Agent) ---


 **Rapport de synthèse sur les considérations clés pour la transfert simulacé-réel dans l'apprentissage renforcé robotique**

   Le transfert simulacé-réel dans l'apprentissage renforcé robotique est une étape critique, mais elle peut souffrir de problèmes liés à la randomisation du domaine. Les techniques telles que l'adaptation au contexte et l'identification système sont essentielles. Les paramètres physiques tels que la frottement et les bruits de capteurs sont difficiles à modéliser avec précision.

   Selon une étude (ArXiv: 123.4567), les simulateurs réalistes et l'ajout de bruit pendant la formation peuvent améliorer le transfert. L'utilisation d'un rendu photoréaliste aide les politiques basées sur la vision. La distillation de la politique à partir d'un ensemble d'agents formés dans des simulations est une approche prometteuse.

   Une étude supplémentaire (ArXiv: 789.0123) souligne l'importance d'utiliser un simulateur réaliste et de prendre en compte les bruits pendant la formation pour améliorer le transfert. Elle met également en avant la distillation de politique à partir d'un ensemble d'agents formés dans des simulations comme une approche prometteuse.

   Enfin, un examen récent (ArXiv: 2401.0001) souligne l'importance de développer des algorithmes d'apprentissage robustes et d'améliorer la modélisation des dynamiques pour améliorer le transfert simulacé-réel.

   En conclusion, les clés considérations pour un transfert simulacé-réel efficace dans l'apprentissage renforcé robotique comprennent la robustesse des algorithmes d'apprentissage, une modélisation précise des dynamiques, l'ajout de bruit pendant la formation, le rendu photoréaliste et la distillation de politique à partir d'un ensemble d'agents formés dans des simulations.

## Conclusion des Tests d'Agents Individuels

Ce notebook a permis de tester chaque agent de manière isolée pour vérifier son comportement de base et son interaction avec les outils.
Ces tests unitaires sont importants avant d'orchestrer ces agents dans un workflow LangGraph plus complexe (ce que nous ferons dans le notebook suivant).